# Generating Maps with Python

## Introduction

What is nice about **Folium** is that it was developed for the sole purpose of visualizing geospatial data. While other libraries are available to visualize geospatial data, such as **plotly**, they might have a cap on how many API calls you can make within a defined time frame. **Folium**, on the other hand, is completely free.


# Exploring Datasets with *pandas* and Matplotlib<a id="0"></a>

Toolkits: This lab heavily relies on [*pandas*](http://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01) and [*Numpy*](http://www.numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01) for data wrangling, analysis, and visualization. The primary plotting library we will explore in this lab is [**Folium**](https://github.com/python-visualization/folium/).

Datasets:

1.  San Francisco Police Department Incidents for the year 2016 - [Police Department Incidents](https://data.sfgov.org/Public-Safety/Police-Department-Incidents-Previous-Year-2016-/ritf-b9ki?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01) from San Francisco public data portal. Incidents derived from San Francisco Police Department (SFPD) Crime Incident Reporting system. Updated daily, showing data for the entire year of 2016. Address and location has been anonymized by moving to mid-block or to an intersection.

2.  Immigration to Canada from 1980 to 2013 - [International migration flows to and from selected countries - The 2015 revision](http://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.shtml?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01) from United Nation's website. The dataset contains annual data on the flows of international migrants as recorded by the countries of destination. The data presents both inflows and outflows according to the place of birth, citizenship or place of previous / next residence both for foreigners and nationals. For this lesson, we will focus on the Canadian Immigration data


# Downloading and Prepping Data <a id="2"></a>


Import Primary Modules:


In [133]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [134]:
file = "C:/Users/micho/OneDrive/Desktop/GITHUB/spacex (2).csv"
spacex_df=pd.read_csv(file)
spacex_df.head(4)

,Unnamed: 0,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


The spacex_df DataFrame by the 'Launch Site' column and retrieves the first row for each group. To mark the launch sites on a map, you need to extract relevant columns (such as latitude and longitude) from this DataFrame and then plot these locations using folium.

launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

Task 1: Mark all launch sites on a map

In [135]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df.head()


,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites.
 Let's visualize those locations by pinning them on a map.
 We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [136]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [137]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

You should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame launch_sites

TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

An example of folium.Circle:

folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))

An example of folium.Marker:

folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))

In [138]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [139]:
# Create a circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    location=nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True,
    fill_color='#d35400'
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a marker at NASA Johnson Space Center's coordinate with a text label showing its name
marker = folium.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
)

# Add the circle and marker to the map
site_map.add_child(circle)
site_map.add_child(marker)

# Save the map to an HTML file
site_map.save("nasa_jsc_map.html")

In [140]:
successful_location = [[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302, -80.577356], [28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302, -80.577356], [28.562302,-80.577356]]	
failed_location =[[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302, -80.577356],[28.562302, -80.577356]]

In [141]:
# Define the list of failed locations
# Add a circle for each failed location
for location in failed_location:
    folium.Circle(
        location=location,  # Each location is a single list [lat, lon]
        radius=1000,  # Radius in meters
        color='red',
        fill=True,
        fill_color='red',
        popup='Failed Location'
    ).add_to(site_map)


In [142]:
folium.Marker(
        location=location,
        popup='Failed Location',
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(site_map)
from IPython.display import display
display(site_map)

In [143]:
# Define the list of failed locations
successful_location = [[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356],[28.562302,-80.577356]]	

# Add a circle for each failed location
for location in successful_location:
    folium.Circle(
        location=location,  # Each location is a single list [lat, lon]
        radius=50000,  # Radius in meters
        color='green',
        fill=True,
        fill_color='green',
        popup='Success Location'
    ).add_to(site_map)
    
from IPython.display import display
display(site_map)

In [144]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Add a marker

    # Add a marker
folium.Marker(
        location=location,
        popup='Successful Launch',
        icon=folium.Icon(color='green', icon='ok-sign')
    ).add_to(site_map)

In [145]:
from IPython.display import display
display(site_map)

In [146]:
import pandas as pd

# Sample DataFrame with 'class' column
launch_sites_data = {
    "name": ["Site A", "Site B", "Site C", "Site D"],
    "lat": [28.562302, 28.562302, 28.562302, 28.562302],
    "lon": [-80.577356, -80.577356, -80.577356, -80.577356],
    "class": [1, 0, 1, 0]  # 1 = Successful, 0 = Failed
}
launch_sites_df = pd.DataFrame(launch_sites_data)

# Define a function to determine marker color based on class value
def determine_marker_color(class_value):
    if class_value == 1:
        return 'green'  # Successful launches
    elif class_value == 0:
        return 'red'  # Failed launches
    else:
        return 'gray'  # Default color for unexpected values

# Apply the function to create the 'marker_color' column
launch_sites_df['marker_color'] = launch_sites_df['class'].apply(determine_marker_color)

# Display the updated DataFrame
print(launch_sites_df)


     name        lat        lon  class marker_color
0  Site A  28.562302 -80.577356      1        green
1  Site B  28.562302 -80.577356      0          red
2  Site C  28.562302 -80.577356      1        green
3  Site D  28.562302 -80.577356      0          red


In [147]:
# Initialize MarkerCluster
marker_cluster = MarkerCluster()

# Example DataFrame with 'marker_color' and coordinates
spacex_data = {
    "name": ["Site A", "Site B", "Site C", "Site D"],
    "lat": [28.562302, 28.562302, 28.562302, 28.562302],
    "lon": [-80.577356, -80.577356, -80.577356, -80.577356],
    "class": [1, 0, 1, 0],  # Assuming 1 = Successful, 0 = Failed
    "marker_color": ['green', 'red', 'green', 'red']
}
spacex_df = pd.DataFrame(spacex_data)

# Add markers to the MarkerCluster
for index, record in spacex_df.iterrows():
    # Create and customize Marker
    marker = folium.Marker(
        location=[record['lat'], record['lon']],
        popup=record['name'],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # Add marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add MarkerCluster to the map
site_map.add_child(marker_cluster)

# Save the map to an HTML file
site_map.save("site_map_with_markers.html")

# Display the map (if in Jupyter Notebook)
from IPython.display import display
display(site_map)


In [ ]:

# Sample DataFrame with launch records
spacex_data = {
    "name": ["Site A", "Site B", "Site C", "Site D"],
    "lat": [28.562302, 28.562302, 28.562302, 28.562302],
    "lon": [-80.577356, -80.577356, -80.577356, -80.577356],
    "class": [1, 0, 1, 0],  # 1 = Successful, 0 = Failed
}
spacex_df = pd.DataFrame(spacex_data)

# Initialize the map (set a central point and zoom level)
central_lat, central_lon = 28.562302, -80.577356
site_map = folium.Map(location=[central_lat, central_lon], zoom_start=10)

# Initialize MarkerCluster
marker_cluster = MarkerCluster()

# Add markers to the MarkerCluster
for index, record in spacex_df.iterrows():
    # Determine marker color based on 'class'
    color = 'green' if record['class'] == 1 else 'red'
    
    # Create and customize Marker
    marker = folium.Marker(
        location=[record['lat'], record['lon']],
        popup=record['name'],
        icon=folium.Icon(color='white', icon_color=color)
    )
    # Add marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add MarkerCluster to the map
site_map.add_child(marker_cluster)

# Save the map to an HTML file
site_map.save("launch_records_map.html")

# Display the map (if in Jupyter Notebook)
from IPython.display import display
display(site_map)


[Jolayemi Babatunde](https://www.linkedin.com/in/babatunde-jolayemi-a05312275/)
